In [1]:
using Revise
using HierarchicalDA
using LinearAlgebra
using OrdinaryDiffEq
using Trixi
using PDMats
using BlockArrays
using LinearMaps
using RecursiveArrayTools
import Base: *, size
using FastGaussQuadrature
using IterativeSolvers
using SciMLOperators
using LinearSolve
using SparseArrays
using Statistics

Precompiling HierarchicalDA
  ✓ HierarchicalDA
  1 dependency successfully precompiled in 6 seconds. 499 already precompiled.


In [2]:
Nx = 10
Ny = 5

nodes = gausslegendre(Nx)[1]
PA = PolyAnnil(nodes, 4; istruncated = true);

Ns = size(PA.P, 1)

6

In [3]:
ys = ObsConstraintVector(rand(Ny), rand(Ns))

([0.08915278830699969, 0.19230182483783465, 0.5655725332637251, 0.8639174048633399, 0.527165172089011], [0.47548778708748096, 0.5964663779956864, 0.6927936827082822, 0.6251866565827696, 0.4169954346802448, 0.8694056869069212])

In [41]:
compute_coefficients(0.0, semi)

┌ Error: Failed to revise /Users/mathieu/Documents/HierarchicalDA.jl/src/HierarchicalDA.jl
│   exception =
│    LoadError: UndefVarError: `@showprogress` not defined
│    in expression starting at /Users/mathieu/Documents/HierarchicalDA.jl/src/inviscid_burgers/generate_data.jl:19
└ @ Revise ~/.julia/packages/Revise/bAgL0/src/packagedef.jl:724


LoadError: UndefVarError: `semi` not defined

In [10]:
H = LinearMap(randn(Ny, Nx))
Σϵ = LinearMap(PDiagMat(0.1 .+ rand(Ny)))
Dθ = LinearMap(Diagonal(0.1 .+ rand(Ns)))
Σx = LinearMap(PDMat((x->x*x')(randn(Nx, Nx))+ 10.0*I))
S = LinearMap(PA.P)

6×10 LinearMaps.WrappedMap{Float64} of
  6×10 SparseMatrixCSC{Float64, Int64} with 30 stored entries

In [13]:
Dθ.lmap

6×6 Diagonal{Float64, Vector{Float64}}:
 0.99076   ⋅         ⋅         ⋅         ⋅         ⋅ 
  ⋅       0.179041   ⋅         ⋅         ⋅         ⋅ 
  ⋅        ⋅        0.585926   ⋅         ⋅         ⋅ 
  ⋅        ⋅         ⋅        0.109142   ⋅         ⋅ 
  ⋅        ⋅         ⋅         ⋅        0.652279   ⋅ 
  ⋅        ⋅         ⋅         ⋅         ⋅        0.666558

In [6]:
X = randn(Nx, 20)

CX = EmpiricalCov(X)

EmpiricalCov(10, 20, [1.3699408366094683 0.7868935576984586 … -0.4628734523687668 0.661103388018871; 0.676693310407815 2.562007523660824 … 1.0766688471557155 0.49953570057961716; … ; -1.608098521389502 -0.809022873720813 … 0.3211349548647069 -2.2041375498534643; -1.3942574262675782 0.6271703503737603 … -0.40303376792097134 -0.49026514476450833], [0.3504504755211593, 0.3718942474972036, -0.6393397633272074, 0.08720324013272149, 0.20455156415438047, -0.04636314740221514, -0.25420079384195093, 0.28638086678928565, -0.3215333645383392, -0.3896869948263239])

In [15]:
CX_op = LinearMaps.FunctionMap{Float64,true}((y,x)->mul!(y, CX, x), Nx; issymmetric=true, isposdef=false)

10×10 LinearMaps.FunctionMap{Float64,true}(#7; issymmetric=true, ishermitian=true, isposdef=false)

In [16]:
sys = ObsConstraintSystem(H, S, Dθ, Σϵ, CX_op)
sys_op = LinearMaps.FunctionMap{Float64,true}((y,x)->mul!(y, sys, x), Ny+Ns; issymmetric=true, isposdef=true)

11×11 LinearMaps.FunctionMap{Float64,true}(#9; issymmetric=true, ishermitian=true, isposdef=true)

In [25]:
mul!(constraint(ys), S, x)

6-element Vector{Float64}:
  0.2419766700785397
 -0.0637843192953752
  0.05487693758287113
  0.11048150025766618
 -0.3608803392817713
 -0.7472894820895091

In [26]:
cg(sys_op, test_1)

11-element Vector{Float64}:
  0.003822608902809471
  0.10078296580154027
  0.6046779025294428
  1.1729805736961914
  0.18635101300545562
 -0.6349775657949539
 -0.4195729437814132
 -0.12584611707172225
  0.6271121069845151
  1.0771967601419645
  0.44941576439393083

In [32]:
cg!(test_2, sys_op, test_2)

([-0.0021007333516648713, 0.09721066078771715, 0.6121600044439626, 1.16363306375158, 0.18543366420591106], [-0.6270708947702954, -0.4026712895725421, -0.10294366248232556, 0.6405715883400466, 1.0836288804777525, 0.4472406718525549])

In [15]:
ObsConstraintVector(4, 5)

([0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0])

In [40]:
 S*randn(Nx)

6-element Vector{Float64}:
  2.4989756225694264
 -3.5776557357037757
  2.7339998666768595
 -0.34102170233855955
 -3.1502365358947335
  6.449549425787787

In [24]:
test_1 = randn(Ny+Ns);
test_2 = ObsConstraintVector(test_1[1:Ny], test_1[Ny+1:Ny+Ns])

([1.1080501436243524, 0.7330601740396127, 1.6666426041899507, -0.4982944715702363, -0.7882386314183453], [-0.6239082600922456, -0.07075679945471874, 0.2585153344188139, -0.8831443883395191, 1.593408756238721, 0.5183868624219737])

In [6]:
sys*test

1096-element Vector{Float64}:
       1.597267513605389e6
 -407440.9648352746
  189069.44139422695
      -2.104865658190339e6
 -724703.2977673538
  957722.0458471449
 -977327.0117853598
      -1.5139969716912904e6
  -15471.087936422628
  300456.0848872592
       1.497703981694312e6
  730236.9690084859
       1.115270636800612e6
       ⋮
   34815.41792535287
  -16905.036705056806
   27302.472324766877
  -30859.14714627152
   20938.84668690218
   -5954.262298030299
    1098.8005935960246
  -46559.63214199423
   88699.33372567977
  -69816.48886531737
   36026.392911733536
  -10253.49851030271

In [8]:
cg(sys_op, test; log = true)

([-0.00039110246296669975, 0.003115343751177836, -0.007735627501787999, -0.013815118062827503, 0.0012559267087378526, -0.0031154224140094652, -0.002103266587713417, 0.004572596878025352, -0.0032685974251189824, 0.012507804495123478  …  0.8143627914946362, 0.39808437916460043, 0.01850204895833222, -0.16996822642054127, -0.15870373020232847, -0.08669516889124644, -0.12236537582712465, -0.10227800585273049, -0.026099396365607215, 0.0026532286273062587], Not converged after 1096 iterations.)

In [18]:
struct ObsConstraintSystem 
    Nx::Int64
    Ny::Int64
    Ns::Int64
    H::LinearMap 
    S::LinearMap
    Dθ::LinearMap
    Σϵ::LinearMap
    Σx::LinearMap
    cache_xx::ArrayPartition{Float64}
    cache_ys::ObsConstraintVector
end

function ObsConstraintSystem(H::LinearMap, S::LinearMap, 
                             Dθ::LinearMap, Σϵ::LinearMap, Σx::LinearMap)
    Nx = size(Σx,1)
    Ny = size(H,1)
    Ns = size(S,1)
    
    cache_xx = ArrayPartition(zeros(Nx), zeros(Nx))
    cache_ys = ArrayPartition(zeros(Ny), zeros(Ns))
    
    return ObsConstraintSystem(Nx, Ny, Ns, H, S, Dθ, Σϵ, Σx, cache_xx, cache_ys)
end

function mul!(output::ObsConstraintVector,
              sys::ObsConstraintSystem,
              input::ObsConstraintVector)
    
    y = observation(input)
    s = constraint(input)
    
    @unpack H, S, Dθ, Σϵ, Σx = sys
#     #############
#     # Compute Hᵀy
#     mul!(sys.cache_xx.x[1], H', y)
    
#     # Compute Sᵀs
#     mul!(sys.cache_xx.x[2], S', s)
        
#     #############
#     # Compute Σx(Hᵀy)
#     mul!(sys.cache_xx.x[1], Σx, sys.cache_xx.x[1])
    
#     # Compute HΣx(Sᵀs)
#     mul!(sys.cache_xx.x[2], Σx, sys.cache_xx.x[2])
    
    #############
    # Compute H Σx(Hᵀy)

    output.x[1] .= Σϵ*y
    output.x[1] .+= H*(Σx*(H'*y))
    output.x[1] .+= H*(Σx*(S'*s))
        
    output.x[2] .= S*(Σx*(H'*y))
    output.x[2] .+= Dθ*s
    output.x[2] .+= S*(Σx*(S'*s))   

    return output
end

function (*)(sys::ObsConstraintSystem, input::ObsConstraintVector)
    output = similar(input)
    mul!(output,sys,input)
    return output
end


function mul!(output::Vector{Float64},
              sys::ObsConstraintSystem,
              input::Vector{Float64})
    
    idx_y = 1:Ny
    idx_s = Ny+1:Ny+Ns
    
    y = view(input, idx_y)
    s = view(input, idx_s)
    
    @unpack H, S, Dθ, Σϵ, Σx = sys
#     #############
#     # Compute Hᵀy
#     mul!(sys.cache_xx.x[1], H', y)
    
#     # Compute Sᵀs
#     mul!(sys.cache_xx.x[2], S', s)
        
#     #############
#     # Compute Σx(Hᵀy)
#     mul!(sys.cache_xx.x[1], Σx, sys.cache_xx.x[1])
    
#     # Compute HΣx(Sᵀs)
#     mul!(sys.cache_xx.x[2], Σx, sys.cache_xx.x[2])
    
    #############
    # Compute H Σx(Hᵀy)

    output[idx_y] .= Σϵ*y
    output[idx_y] .+= H*(Σx*(H'*y))
    output[idx_y] .+= H*(Σx*(S'*s))
        
    output[idx_s] .= S*(Σx*(H'*y))
    output[idx_s] .+= Dθ*s
    output[idx_s] .+= S*(Σx*(S'*s))   

    return output
end

function (*)(sys::ObsConstraintSystem, input::Vector{Float64})
    output = similar(input)
    mul!(output,sys,input)
    return output
end

size(sys::ObsConstraintSystem) = (sys.Ny+sys.Ns, sys.Ny+sys.Ns) 

size (generic function with 504 methods)

In [20]:
sys = ObsConstraintSystem(H, S, Dθ, Σϵ, Σx)

CanonicalIndexError: CanonicalIndexError: getindex not defined for ObsConstraintSystem

In [5]:
MatrixOperator(sys, zeros(Ny+Ns), zeros(Ny+Ns))

LoadError: UndefVarError: `sys` not defined

In [34]:
function apply!(output::Vector{Float64}, sys::ObsConstraintSystem, input::Vector{Float64})
    idx_y = 1:Ny
    idx_s = Ny+1:Ny+Ns
    
    y = view(input, idx_y)
    s = view(input, idx_s)
    
    @unpack H, S, Dθ, Σϵ, Σx = sys
#     #############
#     # Compute Hᵀy
#     mul!(sys.cache_xx.x[1], H', y)
    
#     # Compute Sᵀs
#     mul!(sys.cache_xx.x[2], S', s)
        
#     #############
#     # Compute Σx(Hᵀy)
#     mul!(sys.cache_xx.x[1], Σx, sys.cache_xx.x[1])
    
#     # Compute HΣx(Sᵀs)
#     mul!(sys.cache_xx.x[2], Σx, sys.cache_xx.x[2])
    
    #############
    # Compute H Σx(Hᵀy)

    output[idx_y] .= Σϵ*y
    output[idx_y] .+= H*(Σx*(H'*y))
    output[idx_y] .+= H*(Σx*(S'*s))
        
    output[idx_s] .= S*(Σx*(H'*y))
    output[idx_s] .+= Dθ*s
    output[idx_s] .+= S*(Σx*(S'*s))   

    return output
end

apply! (generic function with 1 method)

In [29]:
LinearMaps.WrappedFunction(sys)

LoadError: UndefVarError: `WrappedFunction` not defined

In [32]:
FunctionOperator(sys)

LoadError: MethodError: no method matching FunctionOperator(::ObsConstraintSystem)

[0mClosest candidates are:
[0m  FunctionOperator(::Any, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m, [91m::Type{iType}[39m, [91m::Type{oType}[39m) where {iType, oType}
[0m[90m   @[39m [35mSciMLOperators[39m [90m~/.julia/packages/SciMLOperators/Kto0y/src/[39m[90m[4mfunc.jl:27[24m[39m
[0m  FunctionOperator(::Any, [91m::AbstractArray[39m; ...)
[0m[90m   @[39m [35mSciMLOperators[39m [90m~/.julia/packages/SciMLOperators/Kto0y/src/[39m[90m[4mfunc.jl:187[24m[39m
[0m  FunctionOperator(::Any, [91m::AbstractArray[39m, [91m::AbstractArray[39m; op_adjoint, op_inverse, op_adjoint_inverse, p, t, accepted_kwargs, T, isinplace, outofplace, has_mul5, isconstant, islinear, isconvertible, batch, ifcache, cache, opnorm, issymmetric, ishermitian, isposdef)
[0m[90m   @[39m [35mSciMLOperators[39m [90m~/.julia/pa

In [19]:
n = 100
A = rand(n, n)
A = A + A' + 2*n*I
b = rand(n)
x = cg(A, b)

100-element Vector{Float64}:
  0.0030407095762021146
  0.003549191465762482
  0.002579108672097998
 -8.285399104774769e-5
 -0.00018301581585213956
  0.0022237227404554435
  0.002025519616156252
  0.0021016976305019653
  0.0028991471777648123
  0.0029124419853746013
  0.0006068077295321253
 -0.0006031853553029227
  0.00045182516005597086
  ⋮
  0.0013346725787940486
 -0.0005213865737275354
  0.00037714667135322884
  0.001620876046806582
  0.001611851547494696
  0.0017718510531242538
 -0.000434449719318406
 -0.0007279131660973888
  0.00016436086055573323
  0.0012760246606995545
  0.00015925831193610353
  0.0018365456542898829

In [20]:
out = randn(Ny+Ns) 

1096-element Vector{Float64}:
 -1.5847559866307692
  1.222262936244968
 -1.2790273456361687
 -0.4374968598391522
  1.099922944962484
 -0.36556102306045
  0.551091548093427
 -1.4146610899398513
 -0.19865444836298274
 -0.3788489430489827
 -0.7364476380163263
  0.010129848901427724
 -0.6366388211656948
  ⋮
  1.27837292225789
 -1.780378657746136
 -2.0866975020084775
 -2.2427724510274563
  1.3135224712171925
  0.8820556126983615
 -0.30384043823219103
 -0.07781018248798904
  1.4924271908871491
  0.37183671334605845
  0.9298279702963536
  1.2566897090047338

In [21]:
cg(sys, out)#; kwargs...)

LoadError: MethodError: no method matching one(::Type{Any})

[0mClosest candidates are:
[0m  one(::Type{Union{Missing, T}}) where T
[0m[90m   @[39m [90mBase[39m [90m[4mmissing.jl:105[24m[39m
[0m  one([91m::Type{Union{}}[39m, Any...)
[0m[90m   @[39m [90mBase[39m [90m[4mnumber.jl:349[24m[39m
[0m  one([91m::Type{VectorInterface.Zero}[39m)
[0m[90m   @[39m [32mVectorInterface[39m [90m~/.julia/packages/VectorInterface/SbpIu/src/[39m[90m[4monezero.jl:61[24m[39m
[0m  ...


In [168]:
@time sys*sys.cache_ys

  0.003500 seconds (27 allocations: 99.125 KiB)


([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

In [169]:
test = ObsConstraintVector(randn(Ny), randn(Ns))

([-1.4912098373553593, -0.2696104357662594, -0.2819347358855617, -0.9262086429757528, -0.09327565596068517, -0.6668734492901138, 1.0682551176108892, 1.0777069312854481, 0.4412579081116321, -0.21110797458569552  …  -0.6510007312086454, 0.7220190726797509, -0.11826292472610035, -0.2653069400392199, -0.3470333560764768, 0.3618988615678917, 1.3069007252501534, -1.0194002780840985, 1.0771860447867967, -0.5358711266189985], [-1.7378561251076985, -1.0247757520788754, 1.4047705495057632, -1.5175879882532095, 0.46229045727226503, 0.6802336513287508, -0.6591789525751265, 0.24354259697489275, -1.1488366669430128, -0.5908398873887823  …  -1.3416785998380587, -1.1251635481836217, -1.4268126186755874, 0.5676322931432145, -0.3679173613958417, 0.9782502248645302, 0.8061450475927457, -1.1969543438203631, -0.895288773532817, -1.043502772988509])

In [43]:
advection_velocity = 1.0
equations = LinearScalarAdvectionEquation1D(advection_velocity)

###############################################################################
#  setup the GSBP DG discretization that uses the Gauss operators from Chan et al.

surface_flux = FluxLaxFriedrichs()
# surface_flux = flux_lax_friedrichs

# volume_integral = VolumeIntegralShockCapturingHG(indicator_sc;
#                                                  volume_flux_dg = surface_flux,
#                                                  volume_flux_fv = surface_flux)
dg = DGMulti(polydeg = 3,
             element_type = Line(),
             approximation_type = GaussSBP(),
             surface_integral = SurfaceIntegralWeakForm(surface_flux),
             volume_integral = VolumeIntegralWeakForm())

###############################################################################
#  setup the 1D mesh

cells_per_dimension = (32,)
mesh = DGMultiMesh(dg, cells_per_dimension,
                   coordinates_min = (-1.0,), coordinates_max = (1.0,),
                   periodicity = true)

###############################################################################
#  setup the semidiscretization and ODE problem

semi = SemidiscretizationHyperbolic(mesh,
                                    equations,
                                    initial_condition_convergence_test,
                                    dg)

tspan = (0.0, 2.0)
ode = semidiscretize(semi, tspan)

###############################################################################
#  setup the callbacks

# prints a summary of the simulation setup and resets the timers
summary_callback = SummaryCallback()

# analyse the solution in regular intervals and prints the results
analysis_callback = AnalysisCallback(semi, interval = 100, uEltype = real(dg))

# handles the re-calculation of the maximum Δt after each time step
stepsize_callback = StepsizeCallback(cfl = 0.75)

# collect all callbacks such that they can be passed to the ODE solver
callbacks = CallbackSet(summary_callback, analysis_callback, stepsize_callback)

###############################################################################
# run the simulation

sol = solve(ode, CarpenterKennedy2N54(williamson_condition = false),
            dt = 1.0, save_everystep = false, callback = callbacks);

# Print the timer summary
summary_callback()


████████╗██████╗ ██╗██╗  ██╗██╗
╚══██╔══╝██╔══██╗██║╚██╗██╔╝██║
   ██║   ██████╔╝██║ ╚███╔╝ ██║
   ██║   ██╔══██╗██║ ██╔██╗ ██║
   ██║   ██║  ██║██║██╔╝ ██╗██║
   ╚═╝   ╚═╝  ╚═╝╚═╝╚═╝  ╚═╝╚═╝

┌──────────────────────────────────────────────────────────────────────────────────────────────────┐
│ SemidiscretizationHyperbolic                                                                     │
│ ════════════════════════════                                                                     │
│ #spatial dimensions: ………………………… 1                                                                │
│ mesh: ………………………………………………………………… Trixi.Affine DGMultiMesh with NDIMS = 1.                         │
│ equations: …………………………………………………… LinearScalarAdvectionEquation1D                                  │
│ initial condition: ……………………………… initial_condition_convergence_test                               │
│ boundary conditions: ………………………… Trixi.BoundaryConditionPeriodic                                  

In [22]:
similar(ObsConstraintVector(rand(10), rand(10)))

([2.51269703e-314, 2.251527859e-314, 2.5126970856e-314, 2.403145989e-314, 2.5704343796e-314, 2.163125329e-314, 1.56416e-319, NaN, 5.0e-324, 2.234869295e-314], [3.5e-323, 5.0e-323, 5.4e-323, 6.4e-323, 7.4e-323, 8.0e-323, 1.14e-322, 1.2e-322, 1.24e-322, 1.33e-322])

In [ ]:
struct SaddleSystem{T,Ns,Nc,TU,TF,TS<:SchurSolverType}
    A :: LinearMap{T}
    B₂ :: LinearMap{T}
    B₁ᵀ :: LinearMap{T}
    C :: LinearMap{T}
    A⁻¹ :: LinearMap{T}
    C⁻¹ :: LinearMap{T}
    A⁻¹B₁ᵀf :: Vector{T}
    B₂A⁻¹r₁ :: Vector{T}
    _f_buf :: Vector{T}
    _u_buf :: Vector{T}
    P :: LinearMap{T}
    S :: LinearMap{T}
    S⁻¹ :: LinearMap{T}
end

In [11]:
h(x) = x[1:2:end]

H = LinearMap(h, 64, 128; issymmetric = false, ismutating = false)

64×128 LinearMaps.FunctionMap{Float64,false}(h; issymmetric=false, ishermitian=false, isposdef=false)

In [10]:
S = LinearMap(PA)

LoadError: UndefVarError: `PA` not defined

In [8]:
typeof(S) <: LinearMap

┌ Error: Failed to revise /Users/mathieu/Documents/HierarchicalDA.jl/src/PA/PA.jl
│   exception =
│    UndefVarError: `T` not defined
│    Stacktrace:
│     [1] top-level scope
│       @ ~/Documents/HierarchicalDA.jl/src/PA/PA.jl:5
│    Revise evaluation error at /Users/mathieu/Documents/HierarchicalDA.jl/src/PA/PA.jl:5
│    
└ @ Revise ~/.julia/packages/Revise/bAgL0/src/packagedef.jl:722


LoadError: UndefVarError: `S` not defined

In [27]:
function strangmul!(C, B)
    C[1] = -2B[1] + B[2]
    for i in 2:length(B)-1
        C[i] = B[i-1] - 2B[i] + B[i+1]
    end
    C[end] = B[end-1] - 2B[end]
    return C
end
using LinearMaps
A = LinearMap(strangmul!, 10; issymmetric=true, ismutating=true)
b = rand(10)

# using IterativeSolvers, LinearAlgebra
# U = gmres(A, b)
# norm(A*U - b)

10-element Vector{Float64}:
 0.3162766737950068
 0.5041365888007923
 0.18470922422961966
 0.4500511821091693
 0.8278905145471707
 0.25922028745628234
 0.8839570719864376
 0.721052864718504
 0.05312059449673312
 0.9027981226142374

In [ ]:
function mul!()

In [29]:
A*b

10-element Vector{Float64}:
 -0.12841675878922132
 -0.5072872795769581
  0.5847693224507222
  0.11249737455845177
 -0.9465095595288897
  1.1934070116210436
 -0.7876409917980888
 -0.5050280629538374
  1.5176097983392753
 -1.7524756507317418

In [52]:
u = ArrayPartition(rand(10), rand(5))

([0.6518576029459328, 0.12515364720407862, 0.9238993155287828, 0.8336727566370029, 0.8584140316519395, 0.03024335269907119, 0.27941151429392697, 0.31146345765012906, 0.1240761192610329, 0.9594262137061715], [0.5009423878888452, 0.6371528448313901, 0.18255707907483432, 0.9381207354254828, 0.7588264695937327])

In [55]:
ObsConstraintVector(randn(10), rand(5))

┌ Error: Failed to revise /Users/mathieu/Documents/HierarchicalDA.jl/src/HierarchicalDA.jl
│   exception =
│    ArgumentError: invalid type for argument u in method definition for observation at /Users/mathieu/Documents/HierarchicalDA.jl/src/update_x/vectors.jl:6
│    Stacktrace:
│     [1] top-level scope
│       @ ~/Documents/HierarchicalDA.jl/src/update_x/vectors.jl:6
│    Revise evaluation error at /Users/mathieu/Documents/HierarchicalDA.jl/src/update_x/vectors.jl:6
│    
└ @ Revise ~/.julia/packages/Revise/bAgL0/src/packagedef.jl:722


([-1.2570173555617543, 0.5202494107448322, 0.16316563288261268, -1.0681720422506906, -0.3654495844240316, 0.3467889610300921, -1.5870616935753792, -0.7323214461925134, 1.2861224755649558, 1.58502014555896], [0.37403391667364905, 0.7190441095726915, 0.5819910537757993, 0.5657090473553854, 0.7316718658539454])

In [54]:
u.x

([0.6518576029459328, 0.12515364720407862, 0.9238993155287828, 0.8336727566370029, 0.8584140316519395, 0.03024335269907119, 0.27941151429392697, 0.31146345765012906, 0.1240761192610329, 0.9594262137061715], [0.5009423878888452, 0.6371528448313901, 0.18255707907483432, 0.9381207354254828, 0.7588264695937327])